In [5]:
import pandas as pd

In [6]:
data = pd.read_csv('https://raw.githubusercontent.com/manthangandhi/hotel_cancellation_prediction/refs/heads/main/data/hotel_bookings.csv')

In [7]:
data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


In [8]:
data.columns.to_list()

['hotel',
 'is_canceled',
 'lead_time',
 'arrival_date_year',
 'arrival_date_month',
 'arrival_date_week_number',
 'arrival_date_day_of_month',
 'stays_in_weekend_nights',
 'stays_in_week_nights',
 'adults',
 'children',
 'babies',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'is_repeated_guest',
 'previous_cancellations',
 'previous_bookings_not_canceled',
 'reserved_room_type',
 'assigned_room_type',
 'booking_changes',
 'deposit_type',
 'agent',
 'company',
 'days_in_waiting_list',
 'customer_type',
 'adr',
 'required_car_parking_spaces',
 'total_of_special_requests',
 'reservation_status',
 'reservation_status_date']

In [24]:
data['reservation_status'].value_counts()

reservation_status
Check-Out    75166
Canceled     43017
No-Show       1207
Name: count, dtype: int64

In [30]:
data['is_canceled'].value_counts()

is_canceled
0    75166
1    44224
Name: count, dtype: int64

In [32]:
data_2017 = data[data['arrival_date_year'] == 2017]

In [33]:
hotel_data_cleaned = data_2017.drop(columns=['reservation_status_date', 'reservation_status','company', 
                                        'agent', 'arrival_date_day_of_month', 'arrival_date_year', 
                                        'arrival_date_week_number', 'assigned_room_type'])

In [34]:
hotel_data_cleaned.dropna()

,hotel,is_canceled,lead_time,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,...,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
9775,Resort Hotel,1,74,January,1,0,2,0.0,0,BB,...,0,0,A,0,No Deposit,0,Transient,57.60,0,0
9776,Resort Hotel,1,62,January,2,2,2,0.0,0,BB,...,0,0,A,0,No Deposit,0,Transient,51.50,0,1
9777,Resort Hotel,1,62,January,2,2,2,0.0,0,BB,...,0,0,A,0,No Deposit,0,Transient,51.50,0,1
9778,Resort Hotel,1,62,January,2,2,2,0.0,0,BB,...,0,0,A,0,No Deposit,0,Transient,51.50,0,1
9779,Resort Hotel,1,71,January,2,2,1,0.0,0,BB,...,0,0,A,0,No Deposit,0,Transient,32.22,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,August,2,5,2,0.0,0,BB,...,0,0,A,0,No Deposit,0,Transient,96.14,0,0
119386,City Hotel,0,102,August,2,5,3,0.0,0,BB,...,0,0,E,0,No Deposit,0,Transient,225.43,0,2
119387,City Hotel,0,34,August,2,5,2,0.0,0,BB,...,0,0,D,0,No Deposit,0,Transient,157.71,0,4
119388,City Hotel,0,109,August,2,5,2,0.0,0,BB,...,0,0,A,0,No Deposit,0,Transient,104.40,0,0


In [41]:
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [42]:
target_dist = alt.Chart(hotel_data_cleaned).mark_bar().encode(
    x='is_canceled',
    y='count()'
)

target_dist

ImportError: The "vegafusion" data transformer and chart.transformed_data feature requires
version 2.0.3 or greater of the 'vegafusion' package.
This can be installed with pip using:
    pip install "vegafusion>=2.0.3"
or conda:
    conda install -c conda-forge "vegafusion>=2.0.3"

ImportError: vegafusion

alt.Chart(...)